# 1) Preparación previa

### Carga de librerías

In [ ]:
import ipyleaflet
import mpu
import pandas as pd
import re
import numpy as np
from ipyleaflet import Map, basemaps, Marker, AwesomeIcon, Icon, FullScreenControl
import seaborn as sns
import matplotlib.pyplot as plt

### Funciones a usar luego

In [ ]:
# La siguiente función nos permite definir la distancia en latitud y longitud respecto de los puentes:
def distancia(lat, long):
    aux = []
    for index, row in puentes_pilar.iterrows():
        aux.append(
            mpu.haversine_distance(
                (row['Latitud'], row['Longitud']), 
                (lat, long)
            )
        )
    return min(aux)

# La siguiente función nos permite aplicar íconos a los marcadores según su quantil:
def marca_propiedad(lat, long, quantile):
    if quantile == 0:
        icono_prop = icon0
    elif quantile == 1:
        icono_prop = icon1
    elif quantile == 2:
        icono_prop = icon2  
    else: 
        icono_prop = icon3
    marker = Marker(location=(lat,long), draggable=False, icon=icono_prop)
    basic_map.add_layer(marker)

### Lectura del dataset limpiado en la notebook de análisis

In [ ]:
data = pd.read_csv('DF_Final.csv')
data.head()

### Filtro del DF

In [ ]:
# Máscara para obtener los pertenecientes a Pilar
mascara_pilar = data.Partido == 'Pilar'
df_solo_pilar = data.loc[mascara_pilar, :]
df_solo_pilar.head()

In [ ]:
# Máscara para obtener las viviendas que no son countries
pilar_partido = df_solo_pilar.isin({'Barrios':['Pilar', 'Del Viso', 'Derqui', 'Villa Rosa', 'Villa Stolfi', 'Manuel Alberti', 'Manzanares'] })
pilar_sincountry = df_solo_pilar[pilar_partido['Barrios']]
pilar_sincountry.head()

In [ ]:
# Máscara para obtener las viviendas que son countries
pilar_country = df_solo_pilar[~pilar_partido ['Barrios']]
pilar_country.head()

In [ ]:
# Máscara para eliminar valores menores a 1
precio_pilar_sin_country = pilar_sincountry['price_aprox_usd'] > 0 # or np.isNan
df_ppsc = pilar_sincountry.loc[precio_pilar_sin_country, :]
# Verificación de valores NaN -> ppsc = partido pilar sin country
df_ppsc.sort_values(by = 'price_aprox_usd', ascending = False)

### Eliminación de los valores nulos

In [ ]:
df_ppsc_latlon = df_ppsc.dropna(subset=['lat', 'lon'])
df_ppsc_latlon.shape

In [ ]:
# Se procede también con la columna 'Precio_USD_por_M2'
df_ppsc_latlon_con_precio = df_ppsc_latlon[df_ppsc_latlon['Precio_USD_por_M2'].notna()]
df_ppsc_latlon_con_precio

# 2) Análisis del DF sin countries

### Definición de cuantiles

In [ ]:
df_ppsc_latlon_con_precio['quantile'] = pd.qcut(df_ppsc_latlon_con_precio['Precio_USD_por_M2'], 4, labels=False)
df_ppsc_latlon_con_precio.round(3)

### Métricas generales del DF sin countries

In [ ]:
medidas_ppsc = df_ppsc_latlon_con_precio[["Precio_USD_por_M2"]].describe().round(2)
medidas_ppsc

### Agrupación según barrios

In [ ]:
df_ppsc_agrup = df_ppsc_latlon_con_precio.groupby('Barrios')
df_ppsc_agrup

### Métricas del DF agrupado sin countries

In [ ]:
medidas_ppsc_agrup = df_ppsc_agrup[["Precio_USD_por_M2"]].describe().round(2)
medidas_ppsc_agrup

In [ ]:
# Orden según la media
medidas_ppsc_agrup.sort_values(by=[('Precio_USD_por_M2', 'mean')], ascending = False)

# 3) Preparación del DF con countries

### Eliminación de valores inconsistentes

In [ ]:
# Máscara para eliminar valores menores a 1 | ppc = partido pilar countries
precio_pilar_country = pilar_country['price_aprox_usd'] > 1
df_ppc = pilar_country.loc[precio_pilar_country, :]
# Verificación de valores NaN
df_ppc.sort_values(by = 'price_aprox_usd', ascending = False)

In [ ]:
# Eliminación de los nulos de la columna LAT y LON
df_ppc_latlon = df_ppc.dropna(subset=['lat', 'lon'])
df_ppc_latlon.shape

### Creación de nuevo DF con puentes

In [ ]:
# Ubicación de los puentes de la panamericana - ramal pilar
puentes_pilar = pd.DataFrame({
    'Nombre': ['Separacion', 'Tortugas', '26', 'Camaño', 'Palmas', '25', 'Manzanares','6'],
    'Latitud': [-34.468040, -34.448354, -34.440682, -34.436667, -34.443766, -34.449550, -34.441083, -34.395740],
    'Longitud': [-58.685014, -58.748577, -58.782740, -58.828145, -58.872848, -58.905720, -58.966898, -59.022886]
})

In [ ]:
print(puentes_pilar)

### Cálculo de la distancia hacia los puentes

In [ ]:
%%capture output
# Cálculo d la distancia de las propiedades hacia los puentes en una nueva columna
df_ppc_latlon['distancia'] = df_ppc_latlon.apply(lambda x: distancia(x['lat'], x['lon']), axis=1)
df_ppc_latlon

### Eliminación de los nulos del precio por metro cuadrado

In [ ]:
df_ppc_latlon_con_precio = df_ppc_latlon[df_ppc_latlon['Precio_USD_por_M2'].notna()]
df_ppc_latlon_con_precio

### Eliminación de outliers

In [ ]:
mask_latlon_con_precio = df_ppc_latlon_con_precio['distancia'] < 90
df_ppc_latlon_con_precio= df_ppc_latlon_con_precio.loc[mask_latlon_con_precio, :]

### Eliminación de barrios nulos

In [ ]:
mask_nulos = df_ppc_latlon_con_precio['Barrios'] != ""
df_final_countries = df_ppc_latlon_con_precio.loc[mask_nulos, :]
df_final_countries

# 4) Análisis del DF con countries

### Armado de cuantiles

In [ ]:
df_final_countries['quantile'] = pd.qcut(df_final_countries['Precio_USD_por_M2'], 4, labels=False)
df_final_countries.round(3)

### Métricas generales del DF con countries

In [ ]:
medidas_ppc = df_final_countries[["Precio_USD_por_M2"]].describe().round(2)
medidas_ppc

### Agrupación según barrios

In [ ]:
df_ppc_agrup = df_final_countries.groupby('Barrios')
df_ppc_agrup

### Métricas específicas del DF con countries agrupado

In [ ]:
medidas_ppc_agrup = df_ppc_agrup[["Precio_USD_por_M2"]].describe().round(2)

In [ ]:
# Orden según la media
medidas_ppc_agrup.sort_values(by=[('Precio_USD_por_M2', 'mean')], ascending = False)

# 4) Gráficos

### Preparación de las variables a usar

In [ ]:
distancia_graf = df_ppc_latlon_con_precio['distancia']*1000
distancia_graf = distancia_graf.astype(int)    

Precio_USD_por_M2= np.around(df_ppc_latlon_con_precio['Precio_USD_por_M2']) 
price_aprox_usd= np.around(df_ppc_latlon_con_precio['price_aprox_usd'])

b2=Precio_USD_por_M2.mean()
c2=distancia_graf.mean()
a2=price_aprox_usd.mean()

b3=Precio_USD_por_M2.median()
c3=distancia_graf.median()
a3=price_aprox_usd.median()

a3,b3,c3

### Gráfico bivariable de distancia vs precio por m² y precio total

In [ ]:
with sns.axes_style('dark'):
    g =sns.jointplot( x=distancia_graf, y=Precio_USD_por_M2 , height=10, ratio=3 )
    f= sns.jointplot( x=distancia_graf, y=price_aprox_usd , height=10, ratio=3 )

### Gráfico de quantiles

In [ ]:
Quantile1 = np.around(df_ppc_latlon_con_precio['quantile'])
bx = sns.boxplot(x = Quantile1, y = distancia1)

In [ ]:
ax = sns.boxplot(x = Quantile1, y = Precio_USD_por_M21 )

### Gráfico de cercanía a la panamericana

In [ ]:
basic_map = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=(-34.44349933885334, -58.86776001513485), zoom=12)

icono_puente = Icon(icon_url='http://localhost:8888/files/TrabajoPractico1/icono_puente.png?_xsrf=2%7C2ca70c0c%7C8e467d878f5d4b68523c5f2b28ba085b%7C1607985927', icon_size=[28, 28])

icon0 = AwesomeIcon(
    name='home',
    marker_color='green',
    icon_color='black',
    spin=False
)

icon1 = AwesomeIcon(
    name='home',
    marker_color='beige',
    icon_color='black',
    spin=False
)

icon2 = AwesomeIcon(
    name='home',
    marker_color='orange',
    icon_color='black',
    spin=False
)

icon3 = AwesomeIcon(
    name='home',
    marker_color='red',
    icon_color='black',
    spin=False
)

for index, row in puentes_pilar.iterrows():
    marker = Marker(location=(row['Latitud'],row['Longitud']), draggable=False, icon=icono_puente)
    basic_map.add_layer(marker);

df_final_countries[['lat','lon','quantile']].apply(lambda x: marca_propiedad(x['lat'],x['lon'],x['quantile']),axis=1)
basic_map.add_control(FullScreenControl())

display(basic_map)